In [1]:
import tensorflow as tf
import json
import numpy as np
import re
from typing import Literal, Union
from tqdm import tqdm

2024-12-18 15:48:11.684484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734529691.697757    9634 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734529691.701642    9634 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-18 15:48:11.716021: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
punctuation = re.compile(r"[^\w\s-]")
def preprocess_text(text):
    text = tf.strings.strip(text) # leading/trailing whitespace removal
    text = tf.strings.lower(text)
    text = punctuation.sub("", text.numpy().decode('utf-8')) # punctuation removal
    return text

In [3]:
TRAIN_PATH = '../data/raw/PIZZA_train.json'
DEV_PATH = '../data/raw/PIZZA_dev.json'
TEST_PATH = '../data/raw/PIZZA_test.json'

def load_data(file_path,type: Literal['train', 'dev', 'test']):
    if type == 'train':
        data_train = []
        with open(TRAIN_PATH, 'r') as f:
            for line in f:
                data_train.append(json.loads(line))
        return data_train
    elif type == 'dev':
        with open(DEV_PATH, 'r') as file:
            data_dev = json.load(file)
        return data_dev
    elif type == 'test':
        data_test = []
        with open(TEST_PATH, 'r') as f:
            for line in f:
                data_test.append(json.loads(line))
        return data_test
    else:
        raise ValueError('Invalid type')


In [8]:
# train_data = load_data(TRAIN_PATH,'train')
# dev_data = load_data(DEV_PATH,'dev')
# test_data = load_data(TEST_PATH,'test')
# print(f"Train data: {len(train_data)}")
# print(f"Dev data: {len(dev_data)}")
# print(f"Test data: {len(test_data)}")

Train data: 2456446
Dev data: 348
Test data: 1357


In [4]:
def extract_text(data,type: Literal['train', 'dev', 'test']):
    if type =='train':
        return data['train.SRC'], data['train.EXR'], data['train.TOP'], data['train.TOP-DECOUPLED']
    elif type == 'dev':
        return data['dev.SRC'], data['dev.EXR'], data['dev.TOP'], data['dev.PCFG_ERR']
    elif type == 'test':
        return data['test.SRC'], data['test.EXR'], data['test.TOP'], data['test.PCFG_ERR']
    else:
        raise ValueError('Invalid type')   

In [5]:
sentences_train, explanations_train, topics_train, decoupled_topics_train=map(np.array, zip(*[extract_text(data,'train') for data in load_data(TRAIN_PATH,'train')]))
sentences_dev, explanations_dev, topics_dev, decoupled_topics_dev=map(np.array, zip(*[extract_text(data,'dev') for data in load_data(DEV_PATH,'dev')]))
sentences_test, explanations_test, topics_test, decoupled_topics_test=map(np.array, zip(*[extract_text(data,'test') for data in load_data(TEST_PATH,'test')]))

In [6]:
processed_sentences_train = [
    preprocess_text(sentence) for sentence in tqdm(sentences_train, desc="Processing Train Sentences")
]
processed_sentences_train = np.array(processed_sentences_train)

Processing Train Sentences: 100%|██████████| 2456446/2456446 [04:57<00:00, 8253.95it/s]


In [7]:
np.save('../data/processed/train/processed_sentences_train.npy', processed_sentences_train)

In [8]:
processed_dev_sentences = [
    preprocess_text(sentence) for sentence in tqdm(sentences_dev, desc="Processing Dev Sentences")
]
processed_dev_sentences = np.array(processed_dev_sentences)

Processing Dev Sentences: 100%|██████████| 348/348 [00:00<00:00, 7785.00it/s]


In [9]:
np.save('../data/processed/dev/processed_dev_sentences.npy', processed_dev_sentences)

In [10]:
processed_test_sentences = [
    preprocess_text(sentence) for sentence in tqdm(sentences_test, desc="Processing Test Sentences")
]
processed_test_sentences = np.array(processed_test_sentences)

Processing Test Sentences: 100%|██████████| 1357/1357 [00:00<00:00, 7603.09it/s]


In [11]:
np.save('../data/processed/test/processed_test_sentences.npy', processed_test_sentences)